In [ ]:
# Import Reinforcement Learning Algoirthms
from stable_baselines3 import A2C, DDPG, DQN, HER, PPO, SAC, TD3
from sb3_contrib import ARS, MaskablePPO, QRDQN, RecurrentPPO, TQC, TRPO

# Import Logger and Recorder
from stable_baselines3.common.logger import configure
from stable_baselines3.common.vec_env import VecVideoRecorder

# Import OpenAI Gym & Other Important Libraries
import gymnasium as gym
import time
from typing import Type

# Import Data Analysis & Visualization Libraies
import numpy  as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def test_algorithm(alg: Type['abc.ABCMeta'], env_name : str, timesteps : int):

    """
    Description:
        This function takes an SB3 Reinforcement Learning Algorithm as a parameter 
        and trains the specified OpenAI Gym environment using this algoirthm. The 
        trained model is then executed and a video recording of the agent is saved.

    Parameters:
        - alg (abc.ABCMeta) : Indicates which algorithm to use for training
        - env_name (str)    : Name of the Environment to test in
        - timesteps (int)   : Number of Timesteps for the Learning Stage

    Returns:
        - timesteps (int)      : Number of Timesteps the trained agent performed during execution
        - total_time (float)   : Number of Seconds the Training / Execution Took
        - total_reward (float) : The Total Reward From the Execution Stage of the Trained Agent
    """

    # Start Algorithm Timer
    alg_start = time.time()
    
    # Create environment
    env = None
    try: env = gym.make(env_name, render_mode = "rgb_array")
    except: raise ValueError(f"Unknown Environment: {env_name}")

    # Instantiate the agent
    model = None
    try: model = alg("MlpPolicy", env, verbose = 0)
    except: raise ValueError(f"Unknown Algorithm: {alg}")

    # Set up a new logger
    print("Setting up Logger:")
    log_path = f'./{env_name}/{alg.__name__}/log/'
    formats = ["stdout", "csv", "tensorboard"]
    new_logger = configure(log_path, formats)
    model.set_logger(new_logger)
    
    # Train the agent and display a progress bar
    model.learn(total_timesteps = int(timesteps), progress_bar = True)
    
    # Initialize total reward and timesteps
    total_rew : float = 0
    timesteps : int   = 0
    
    # Create a vectorized environment for recording
    vec_env = VecVideoRecorder(
        venv                 = model.get_env(),
        video_folder         = f'./{env_name}/{alg.__name__}/videos/',
        record_video_trigger = 10_000,
        video_length         = 10_000,
    )
    
    # Execute the Trained Agent
    obs = vec_env.reset()
    while True:
        action, _states = model.predict(obs, deterministic=True)
        obs, rewards, done, info = vec_env.step(action)
        vec_env.render("rgb_array")
        total_rew += rewards
        timesteps += 1
        if done: break
    
    # Close the vectorized Environment - Stops Recording
    vec_env.close()

    # Stop the Algorithm Timer
    total_time = time.time() - alg_start

    # Return the timesteps, execution time, and reward
    return timesteps, total_time, total_rew[0]

    # Delete Local Variables for Next Iteration
    del model, env, vec_env

In [ ]:
#############################
# NON-CHANGEABLE PARAMETERS  
#############################

test_num = 1                       # Tracks the Test # of Current Iteration
results_dict = {}                  # Stores the Result for each Algorithm
it = 0                             # Iterator for the colors list

# Colors for creating data visuals
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']


#############################
# CHANGEABLE PARAMETERS 
#############################

env_name    = "LunarLander-v2"     # Name of the Environment to Run
train_steps = 500_000              # Number of Timesteps for Learning

# RL Algorithms for Testing
rl_algs = [A2C, DQN, PPO, TRPO]

In [ ]:
# Start Test Counter & Start the Program Timer
prog_start = time.time()

# Test each RL algorithm with the defined parameters
for alg in rl_algs:

    # Program Status Output
    print("=============================================")
    print(f"Starting Test #{test_num}")
    print(f"   Environment: {env_name}")
    print(f"   Algorithm:   {alg.__name__}")
    print(f"   Timesteps:   {int(train_steps)}")
    print("=============================================")
    
    # Test the Algorithm with the defined parameters
    total_timesteps, total_time, total_reward = test_algorithm(
        alg       = alg,
        env_name  = env_name,
        timesteps = train_steps
    )

    # Add the Results to the Results Dictionary
    results_dict[alg.__name__] = (f"--------------------\n"
            f"The {alg.__name__} algorithm achieved a total reward of {total_reward:.2f} in {total_timesteps} timesteps.\n"
            f"The {alg.__name__} algorithm took {total_time:.2f} s ({(total_time/60):.2f} min) to execute.\n")

    # Print the Results
    print(results_dict[alg.__name__])

    # Increment the Test Number to start new iteration
    test_num += 1

# Stop the Program Timer
tot_time = time.time() - prog_start

In [ ]:
# Print the Results Dictionary HEADER
print("\n=============================================")
print("                Final Results                ")
print("=============================================\n")

for alg in rl_algs: 
    
    # Print the Result of Each Function
    print(f"{alg.__name__}:\n{results_dict[alg.__name__]}\n")

    # Read the Progress CSV file for each algorithm
    filename = f'./{env_name}/{alg.__name__}/log/progress.csv'
    df = pd.read_csv(filename)

    # Extract the Necessary Columns
    x = df['time/total_timesteps']
    y = df['rollout/ep_rew_mean']

    # Add the Data to the Plot
    plt.plot(x, y, label = alg.__name__, color = colors[it], linestyle='-', marker='o')
    it = ((it + 1) % len(colors))

# Add Labels and Title to the Plot
plt.xlabel('Total Timesteps')
plt.ylabel('Total Mean Reward')
plt.title('Algorithm Performance for Lunar Lander Environment')

# Display the Plot and its Legend
plt.legend()
plt.show()

# Print the Termination Message
print("\n=============================================")
print(f"This program took {(tot_time/60):.2f} mins to execute.\n")
print("Terminating Program...")
print("=============================================\n")